# Auto Insurance Analysis

## Exploratory Data Analysis

## Project Goal

## Summary of Data

### Library Import

In [1]:
#Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import linear_model
%matplotlib inline

## Data Import and Data Exploration

In [2]:
# import auto insurance data
# change values with empty spaces in the dataset to NaN values
auto_insurance_df = pd.read_csv('../data/auto_insurance_data.csv', na_values=' ')
# change column names to lower-case
auto_insurance_df.columns = [i.lower() for i in auto_insurance_df.columns]
# quick overview of the dataset
auto_insurance_df

,target_flag,target_amt,kidsdriv,age,homekids,yoj,income,parent1,home_val,mstatus,...,bluebook,tif,car_type,red_car,oldclaim,clm_freq,revoked,mvr_pts,car_age,urbanicity
0,0,0.0,0,60,0,11,67349.381620,No,0.0000,No,...,14230,11,Minivan,yes,4461,2,No,3,18,Highly Urban/ Urban
1,0,0.0,0,43,0,11,91449.254640,No,257251.6354,No,...,14940,1,Minivan,yes,0,0,No,0,1,Highly Urban/ Urban
2,0,0.0,0,35,1,10,16038.587100,No,124190.7529,Yes,...,4010,4,SUV,no,38690,2,No,3,10,Highly Urban/ Urban
3,1,2946.0,0,34,1,12,125301.242500,Yes,0.0000,No,...,17430,1,Sports Car,no,0,0,No,0,7,Highly Urban/ Urban
4,1,2501.0,0,34,0,10,62977.824160,No,0.0000,No,...,11200,1,SUV,no,0,0,No,0,1,Highly Urban/ Urban
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6040,0,0.0,0,41,0,7,6255.902602,No,0.0000,No,...,5600,1,Pickup,no,0,0,No,0,7,Highly Rural
6041,0,0.0,0,35,0,11,43111.840100,No,0.0000,No,...,27330,10,Panel Truck,yes,0,0,No,0,8,Highly Rural
6042,0,0.0,1,45,2,9,164668.837000,No,386273.4090,Yes,...,13270,15,Minivan,no,0,0,No,2,17,Highly Urban/ Urban
6043,0,0.0,0,50,0,7,43445.497870,No,149247.7801,Yes,...,22550,6,Minivan,no,0,0,No,0,11,Highly Urban/ Urban


In [3]:
# quick review of the variables in the dataset
auto_insurance_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6045 entries, 0 to 6044
Data columns (total 25 columns):
target_flag    6045 non-null int64
target_amt     6045 non-null float64
kidsdriv       6045 non-null int64
age            6045 non-null int64
homekids       6045 non-null int64
yoj            6045 non-null int64
income         6045 non-null float64
parent1        6045 non-null object
home_val       6045 non-null float64
mstatus        6045 non-null object
sex            6045 non-null object
education      6045 non-null object
job            6045 non-null object
travtime       6045 non-null float64
car_use        6045 non-null object
bluebook       6045 non-null int64
tif            6045 non-null int64
car_type       6045 non-null object
red_car        6045 non-null object
oldclaim       6045 non-null int64
clm_freq       6045 non-null int64
revoked        6045 non-null object
mvr_pts        6045 non-null int64
car_age        6045 non-null int64
urbanicity     6045 non-null object


In [5]:
# quick review of the charcteristics of each variable in the dataset
auto_insurance_df.describe()

,target_flag,target_amt,kidsdriv,age,homekids,yoj,income,home_val,travtime,bluebook,tif,oldclaim,clm_freq,mvr_pts,car_age
count,6045.000000,6045.000000,6045.000000,6045.000000,6045.000000,6045.000000,6045.000000,6045.000000,6045.000000,6045.000000,6045.000000,6045.000000,6045.000000,6045.000000,6045.000000
mean,0.265012,1479.662715,0.173201,44.628453,0.743424,10.494624,58177.014531,150102.075243,33.697596,15235.609595,5.360298,4004.875600,0.784119,1.699752,7.920926
std,0.441376,4553.172055,0.515424,8.707805,1.132740,4.138508,43826.980030,123728.723057,15.889058,8040.962717,4.144664,8822.509329,1.153884,2.157899,5.583880
min,0.000000,0.000000,0.000000,16.000000,0.000000,0.000000,0.000000,0.000000,5.000000,1500.000000,1.000000,0.000000,0.000000,0.000000,-3.000000
25%,0.000000,0.000000,0.000000,39.000000,0.000000,9.000000,26748.313360,0.000000,22.803134,9170.000000,1.000000,0.000000,0.000000,0.000000,1.000000
50%,0.000000,0.000000,0.000000,45.000000,0.000000,11.000000,51624.156770,159152.495700,33.023589,14080.000000,4.000000,0.000000,0.000000,1.000000,8.000000
75%,1.000000,1037.000000,0.000000,51.000000,1.000000,13.000000,81287.490030,233052.859700,43.949410,20120.000000,7.000000,4546.000000,2.000000,3.000000,12.000000
max,1.000000,85523.653350,4.000000,81.000000,5.000000,23.000000,367030.262000,885282.345000,142.120630,65970.000000,25.000000,57037.000000,5.000000,13.000000,28.000000


In [4]:
# check the number of NaN values in the dataset
auto_insurance_df.isna().sum()

target_flag    0
target_amt     0
kidsdriv       0
age            0
homekids       0
yoj            0
income         0
parent1        0
home_val       0
mstatus        0
sex            0
education      0
job            0
travtime       0
car_use        0
bluebook       0
tif            0
car_type       0
red_car        0
oldclaim       0
clm_freq       0
revoked        0
mvr_pts        0
car_age        0
urbanicity     0
dtype: int64

## Data Cleaning

## Initial Findings

## Train/Test Dataset Creation